# Simple Incident Response Bot using CHatGPT

## Cyber Incident Analysis Tool

This Python tool analyzes cybersecurity incidents using OpenAI’s GPT model. It processes a user-provided description, assesses severity, recommends recovery steps, and extracts security-related entities like IPs, domains, and emails.The tool queries OpenAI’s API for an assessment and identifies potential risks. If an IP is found, it can optionally retrieve geolocation and ISP details using the IPInfo API.To use, install the required Python libraries, set the OpenAI API key, and run the script. Input a security incident description to receive a severity rating, mitigation steps, and entity details. An IPInfo API key can optionally enrich IP data.Requires an OpenAI API key, with optional IPInfo API integration.

### Citation:

This assignment was completed with assistance from ChatGPT 4.0, developed by OpenAI, for suggestions and validation purposes. The tool was utilized to refine ideas, validate methods, and ensure clarity in the analysis.

### Impoerting Modules & Settting Keys

In [ ]:
# Requires pip install openai
# !pip install ipywidgets

import openai
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import re
import requests
import json

# Set your OpenAI API Key from https://platform.openai.com/signup/
client = openai.OpenAI(api_key="ENTER YOUR KEY")

# Optional: Set your IPInfo API key for entity enrichment from https://ipinfo.io/
IPINFO_API_KEY = "ENTER YOUR KEY"

# In productin we would secure this API key in a secure vault like AWS Secrets Manager or Azure Key Vault

In [4]:

# Create interactive input and buttons
incident_text = widgets.Textarea(
    placeholder="Describe the security incident here...",
    layout=widgets.Layout(width="100%", height="150px")
)

# Need more buttons for the user to interact with the bot
submit_button = widgets.Button(description="Analyze Incident", button_style='primary')
clear_button = widgets.Button(description="Clear Results", button_style='danger')
new_prompt_button = widgets.Button(description="New Prompt", button_style='info')

# Output area for displaying results
output = widgets.Output()

# Function to query OpenAI API for security analysis using the latest API structure.
# Note the system prompt below, this is a ctitical part of groudning the AI model and making adjustments later.
def query_openai(prompt):
    try:
        response = client.chat.completions.create( 
            model="gpt-4-turbo",  # In later rounds, testing with less expensive models would be a good idea.
            messages=[
                {"role": "system", "content": "You are a cybersecurity expert providing severity assessments and recommendations for security incidents."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"⚠️ OpenAI API Error: {str(e)}"

#Function to use regex to extract entities from the incident description, only using IP enrichment in this example.
def extract_entities(text):
    ip_pattern = r"\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b"
    domain_pattern = r"\b(?:[a-zA-Z0-9-]+\.)+[a-zA-Z]{2,}\b"
    email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b"
    hostname_pattern = r"\b(?:[a-zA-Z0-9-]+\.)+[a-zA-Z]{2,}\b"

    return {
        "IP Addresses": list(set(re.findall(ip_pattern, text))),
        "Domains": list(set(re.findall(domain_pattern, text))),
        "Emails": list(set(re.findall(email_pattern, text))),
        "Hostnames": list(set(re.findall(hostname_pattern, text)))
    }

# Retrieves IP details from the IPInfo API.
def get_ip_info(ip):
    if not IPINFO_API_KEY:
        return {"error": "IPInfo API key is not configured."}
    
    try:
        url = f"https://ipinfo.io/{ip}/json?token={IPINFO_API_KEY}"
        response = requests.get(url)
        return response.json()
    except Exception as e:
        return {"error": f"Could not retrieve IP info: {str(e)}"}

# Function for 'Analyze Incident' button handler and formatting the output.
def analyze_incident(b):
    output.clear_output()
    incident = incident_text.value.strip()
    
    if not incident:
        with output:
            display(Markdown("⚠️ **Please enter a security incident description.**"))
        return
    
    # Get OpenAI analysis
    analysis = query_openai(incident)
    
    # Extract security-related entities
    entities = extract_entities(incident)
    
    with output:
        display(Markdown("### 🛡️ **Security Analysis**"))
        display(Markdown(f"**Incident Description:**\n{incident}"))
        display(Markdown(f"**Assessment & Recommendations:**\n{analysis}"))
        
        display(Markdown("### 🔍 **Extracted Entities**"))
        for key, values in entities.items():
            if values:
                display(Markdown(f"**{key}:** {', '.join(values)}"))
        
        # If IP enrichment is enabled, fetch and display IP details
        if entities["IP Addresses"] and IPINFO_API_KEY:
            display(Markdown("### 🌍 **IP Address Information**"))
            for ip in entities["IP Addresses"]:
                ip_info = get_ip_info(ip)
                display(Markdown(f"**IP:** {ip}"))
                display(Markdown(f"```json\n{json.dumps(ip_info, indent=4)}\n```"))

# Clears output and resets input.
def clear_results(b):
    output.clear_output()
    incident_text.value = ""

# Clears only the input field while keeping the results visible.
def new_prompt(b):
    incident_text.value = ""

# Attach button actions
submit_button.on_click(analyze_incident)
clear_button.on_click(clear_results)
new_prompt_button.on_click(new_prompt)

# Display the widgets
display(Markdown("## 🛡️ GenAI Security Response Bot"))
display(incident_text, submit_button, new_prompt_button, clear_button, output)


## 🛡️ GenAI Security Response Bot

Textarea(value='', layout=Layout(height='150px', width='100%'), placeholder='Describe the security incident he…

Button(button_style='primary', description='Analyze Incident', style=ButtonStyle())

Button(button_style='info', description='New Prompt', style=ButtonStyle())

Button(button_style='danger', description='Clear Results', style=ButtonStyle())

Output()

#### Sapmple prompt suggestions:

#### Attempted to sign-in from anonymous IP address 149.102.254.23

#### 'Cryptor' malware was detected

#### Multiverze' hacktool was detected 

#### A potentially malicious URL click was detected, http://help--trezorhelps.webflow.io/